In [3]:
from qdrant_client import QdrantClient, models
from dotenv import load_dotenv
import os
load_dotenv()

QDRANT_API = os.getenv("QDRANT_API")
QDRANT_URL = os.getenv("QDRANT_URL")

client = QdrantClient(location=QDRANT_URL, api_key=QDRANT_API)

In [6]:
collection_name = "first_collection"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=4,
        distance=models.Distance.COSINE
    )
)

True

In [7]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='first_collection')])

In [8]:
points = [
    models.PointStruct(
        id=1,
        vector=[0.1,0.2,0.3,0.4],
        payload={"category": "example"}
    ),
    models.PointStruct(
        id=2,
        vector=[0.1,0.5,-0.3,0.2],
        payload={"category": "demo"}
    )
]

client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
collection_info =client.get_collection(collection_name)
print(collection_info)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> warnings=None indexed_vectors_count=0 points_count=2 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0, wal_retain_cl

In [10]:
query_vector = [0.1,0.5,-0.5,0.3]

search_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=1
)

print(search_results)

points=[ScoredPoint(id=2, version=1, score=0.9716054, payload={'category': 'demo'}, vector=None, shard_key=None, order_value=None)]


In [18]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="category",
    field_schema=models.PayloadSchemaType.KEYWORD
)


UpdateResult(operation_id=9, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:

search_filter = models.Filter(
    must=[
        models.FieldCondition(
            key="category",
            match=models.MatchValue(value="demo")
        )
    ]
)

search_results_filtered = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    query_filter=search_filter,
    limit=1
)

print(search_results_filtered)

points=[ScoredPoint(id=2, version=1, score=0.9716054, payload={'category': 'demo'}, vector=None, shard_key=None, order_value=None)]
